In [1]:
import numpy as np
import cv2 as cv
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn import decomposition
import os
from tqdm import tqdm
import json


In [2]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
model = ResNet50(weights = 'imagenet', include_top = False, pooling="avg")

94781440/94765736 [==============================] - 1s 0us/step


In [4]:
#returns a 2048 feature vector
def get_image_features(orig):
  
  # Convert image to RGB from BGR (another way is to use "image = image[:, :, ::-1]" code)
  orig = cv.cvtColor(orig, cv.COLOR_BGR2RGB)
  # Resize image to 224x224 size
  image = cv.resize(orig, (224, 224)).reshape(-1, 224, 224, 3)
  # We need to preprocess imageto fulfill ResNet50 requirements
  image = preprocess_input(image)
  # Extracting our features
  features = model.predict(image)

  return features



In [24]:
images = os.listdir("/content/drive/MyDrive/NTMC/dataset-8000")

features_list = []


for i in tqdm(range(len(images))):
 
  # Read image
  image = images[i]
  orig = cv.imread(f'/content/drive/MyDrive/NTMC/dataset-8000/{image}')
  image_index = image.split(".")[0]

  features_list.append(get_image_features(orig)[0])

  

100%|██████████| 8000/8000 [33:16<00:00,  4.01it/s]


In [31]:
#reduce 2048 vec dimesnions
#pca
X = features_list
pca = decomposition.PCA(n_components=32)
pca.fit(X)
X = pca.transform(X)


print(len(X))

8000


In [32]:
images = os.listdir("/content/drive/MyDrive/NTMC/dataset-8000")
RESULTS = []

for i in range(len(X)):
  RESULT = {} 
  image = images[i]
  feature_vec = X[i]
  image_index = image.split(".")[0]

  RESULT['id'] = image_index
  RESULT['feature_vec'] = list(feature_vec)
  RESULTS.append(RESULT)

print(len(RESULTS))

8000


In [ ]:
SAVE_DIR = "/content/drive/MyDrive/NTMC/"
SAVE_FILE_NAME = f"features_64-8000.jsonl"
SAVE_PATH = SAVE_DIR + SAVE_FILE_NAME


for row in RESULTS:

    json_object = json.dumps(row)

    with open(SAVE_PATH, "a") as outfile:
        outfile.write(json_object)
        outfile.write('\n')